# galpynostatic tutorial

## Quickstart

A simple screenshot of how to use **galpynostatic** is shown in the following script, where you have a first part of preprocessing of the experimental data and a second part of fitting:
```python
import galpynostatic

# experimental data definition (with numpy and pandas)
C_rates, dataframes, eq_pot, xmax, d = ...

# obtain discharge capacities
xmaxs = galpynostatic.preprocessing.get_discharge_capacities(dataframes, eq_pot)

# xmaxs normalization by a maximum value
xmaxs = xmaxs / xmax

# fit the model
dataset = galpynostatic.datasets.load_spherical()
greg = galpynostatic.model.GalvanostaticRegressor(dataset, d, 3)
greg.fit(C_rates, xmaxs)

# get the diffusion coefficient and the kinetic rate constant
dcoeff = greg.dcoeff_
k0 = greg.k0_

# and an estimation of the characteristic diffusion length to charge the 80%
# of the electrode in 5 minutes
new_d = greg.t_minutes_lenght()
```

The `greg` object also allows to obtain the values predicted by the model, plot these predictions next to the experimental data and plot the points on the surface on which they were fitted.

## Lithium battery systems

In the following section we will work with different systems relevant to the use and research of lithium batteries.

We start by defining the path where the examples data are located

In [1]:
path = "../../examples/data/"

and import all the necessary packages

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

along with the package developed here

In [3]:
import galpynostatic

### Natural graphite

#### Data visualization

#### Preprocessing

#### Fitting

#### Results

### LTO

### LFP

### LCO

### LMO

### LMNO